In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_09b import *
import time
from fastprogress.fastprogress import master_bar, progress_bar
from fastprogress.fastprogress import format_time

In [24]:
!pip show fastprogress

Name: fastprogress
Version: 1.0.0
Summary: A nested progress with plotting options for fastai
Home-page: https://github.com/fastai/fastprogress
Author: Sylvain Gugger
Author-email: info@fast.ai
License: Apache Software License 2.0
Location: /home/sandmann/anaconda3/envs/fastbook/lib/python3.8/site-packages
Requires: numpy
Required-by: fastai


# Imagenette Data

In [3]:
path = untar_data(URLs.IMAGENETTE_160)

In [4]:
tfms = [make_rgb, ResizeFixed(128), to_byte_tensor, to_float_tensor]
bs = 64

il = ImageList.from_files(path, tfms=tfms)
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='val'))
ll = label_by_func(sd, parent_labeler, proc_y=CategoryProcessor())
data = ll.to_databunch(bs, c_in=3, c_out=10, num_workers=4)

In [5]:
nfs = [32] * 4

In [6]:
AvgStatsCallback??

Init signature: AvgStatsCallback(metrics)
Docstring:      <no docstring>
Source:        
class AvgStatsCallback(Callback):
    def __init__(self, metrics):
        self.train_stats, self.valid_stats = AvgStats(metrics, True), AvgStats(metrics, False)

    def begin_epoch(self):
        self.train_stats.reset()
        self.valid_stats.reset()

    def after_loss(self):
        stats = self.train_stats if self.in_train else self.valid_stats
        with torch.no_grad(): stats.accumulate(self.run)

    def after_epoch(self):
        self.logger(self.train_stats)
        self.logger(self.valid_stats)
File:           ~/repo/fastai-course-v3/nbs/dl2/selfmade/exp/nb_09b.py
Type:           type
Subclasses:     


In [12]:
#export
class AvgStatsCallback(Callback):
    def __init__(self, metrics):
        self.train_stats, self.valid_stats = AvgStats(metrics, True), AvgStats(metrics, False)
        
    def begin_fit(self):
        met_names = ['loss'] + [m.__name__ for m in self.train_stats.metrics]
        names = ['epoch'] + [f'train_{n}' for n in met_names] + [f'valid_{n}' for n in met_names] + ['time']
        self.logger(names)
        
    def begin_epoch(self):
        self.train_stats.reset()
        self.valid_stats.reset()
        self.start_time = time.time()
    
    def after_loss(self):
        stats = self.train_stats if self.in_train else self.valid_stats
        with torch.no_grad(): stats.accumulate(self.run)
            
    def after_epoch(self):
        stats = [str(self.epoch)]
        for o in [self.train_stats, self.valid_stats]:
            stats += [f'{v:.6f}' for v in o.avg_stats]
        stats += [format_time(time.time() - self.start_time)]
        self.logger(stats)

In [13]:
#export
class ProgressCallback(Callback):
    _order = -1
    def begin_fit(self):
        self.mbar = master_bar(range(self.epochs))
        self.mbar.on_iter_begin()
        self.run.logger = partial(self.mbar.write, table=True)
        
    def after_fit(self): self.mbar.on_iter_end()
    def after_batch(self): self.pb.update(self.iter)
    def begin_epoch(self): self.set_pb()
    def begin_validate(self): self.set_pb()
        
    def set_pb(self):
        self.pb = progress_bar(self.dl, parent=self.mbar)
        self.mbar.update(self.epoch)

In [14]:
cbfs = [partial(AvgStatsCallback, accuracy),
       CudaCallback, ProgressCallback,
       partial(BatchTransformXCallback, norm_imagenette)]

In [15]:
learn = get_learner(nfs, data, 0.4, conv_layer, cb_funcs=cbfs)

In [16]:
learn.fit(2)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.839150,0.359594,3.395307,0.181401,00:08
1,1.466604,0.503749,2.476956,0.190573,00:08


In [18]:
!python notebook2script.py 09c_add_progress_bar.ipynb

Converted 09c_add_progress_bar.ipynb to exp/nb_09c.py
